<a href="https://colab.research.google.com/gist/VictorPLopes/6dd3fcbc1aa1eb787e41c2f00bf4a4d3/coleta_dados_climaticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Redes de Computadores 1 – Projeto Prático Interdisciplinar**

## Engenharia da Computação – Instituto Federal de Educação, Ciência e Tecnologia de São Paulo, Campus Piracicaba

### Alunos

- [Alexandre Medici Ceregato](mailto:)
- [André Lisboa Augusto](mailto:andre.lisboa@aluno.ifsp.edu.br)
- [Marcos Henrique Maimoni Campanella](mailto:marcos.campanella@aluno.ifsp.edu.br)
- [Rodolfo Henrique Raymundo Engelmann](mailto:rodolfo.engelmann@aluno.ifsp.edu.br)
- [Victor Probio Lopes](mailto:victor.probio@aluno.ifsp.edu.br)

## Introdução

Projeto desenvolvido em conjunto com a matéria de Microcontroladores, ministrada pelo Prof. Pablo Rodrigo de Souza.
O tema deste projeto interdisciplinar foca na coleta e análise de dados climáticos em tempo real, elaborando uma aplicação IoT na qual os aspectos de hardware e software serão desenvolvidos, respectivamente, nas disciplinas de Microcontroladores e Redes de Computadores 1.
</br>
Os dados do tema são coletados no ambiente de cultivo da agricultura de precisão, afim de serem processados adequadamente em prol da tomada de ações que visem melhorar a eficiência geral do plantio.
</br>
Em maiores detalhes, o objetivo do programa desenvolvido é coletar os dados referentes à temperatura, umidade e pressão de um certo local onde se encontra uma placa microcontroladora ESP32 (cujo trabalho é ler os dados e enviá-los para um _broker_ MQTT). Com base nos dados coletados, o programa decide se uma semeadora e/ou uma irrigadora devem ser acionados no local de medição, e envia essas respostas para o _broker_, que são então lidas pelo ESP32, que por sua vez efetua as ações necessárias.
</br>
Além disso, o programa também salva as variáveis climaticas lidas do ESP32 em um arquivo CSV (um segundo por linha). Esse arquivo pode então ser carregado no software novamente para gerar gráficos referentes às medições em um determinado período.

## Materiais e métodos

O presente trabalho foi desenvolvido usando a linguagem de programação Python, funcionando em um _notebook_ Jupyter, programado especificamente para funcionar no serviço de hospedagem de _notebooks_ Google Colaboratory - já que utiliza-se de algumas de suas funcionalidades específicas (como menus e a integração com o Google Drive para hospedagem de arquivos).
</br>
Para o correto funcionamento do software, foi empregado o uso das bibliotecas `paho-mqtt`, `time`, `pandas`, `matplotlib`, `shutil`, `google` e `os`.

## Instruções de uso

Para executar o programa, é necessário executar suas células (partes individuais de código em um _notebook_ Jupyter). Cada Célula possui instruções específicas que podem ser lidas em uma célula textual acima do código. Para executar uma célula, basta mover o _mouse_ para cima desta e clicar sobre o botão de execução, em seu canto superior esquerdo.

### Importação das bibliotecas

É necessário executar a célula abaixo para importar as bibliotecas utilizadas.

In [ ]:
# BIBLIOTECAS

!pip install paho-mqtt
import paho.mqtt.client as mqtt
import time
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from google.colab import files, drive
import os.path

### Definição das funcões do programa

A célula abaixo contém as instruções necessárias para definir as rotinas utilizadas ao longo do programa. É a parte responsável por se conectar ao _broker_ MQTT escolhido (_broker_ público do HiveMQ), se inscrever nos tópicos de interesse, receber os dados do ESP32, salva-los em um arquivo CSV, tomar as decisões necessárias e enviar as respostas de volta ao _broker_. É necessário executa-la uma vez.

In [ ]:
# FUNÇÕES

# Nome do arquivo CSV gerado
file_name = 'dadosClimaticosGrupoZ.csv'

# Dados da conexão ao broker
broker_address = "broker.hivemq.com"
port = 1883
username = "dadosClimaticosGrupoZ"
password = ""

# Função que envia os dados de irrigação e semeadura
def data_sender(irrigacao, semeadura):
    client = mqtt.Client()

    # Função executada ao se conectar
    def on_connect(client, userdata, flags, rc):
        print("Conectado com código de resultado: " + str(rc))
        client.subscribe("GrupoZ_irrigacao")
        client.publish("GrupoZ_irrigacao", irrigacao, retain=True)
        client.subscribe("GrupoZ_semeadura")
        client.publish("GrupoZ_semeadura", semeadura, retain=True)
    client.on_connect = on_connect

    # Dados de conexão
    client.username_pw_set(username, password)

    # Se conecta ao broker
    client.connect(broker_address, port, 60)

    # Tenta se conectar
    try:
        # Entra em loop
        client.loop_start()
        time.sleep(2)
    # Desconecta em caso de erros
    except:
        client.disconnect()
        raise Exception("Desconectando...")

# Função que salva os dados no CSV
def salvar_para_csv(dados):
    with open(file_name, 'a', newline='\n') as arquivo_csv:
        arquivo_csv.write(dados + '\n')
    print("Dados salvos no arquivo CSV com sucesso!")

# Função que trata os dados lidos
def data_treatment(dados):
    # Salva os dados lidos
    salvar_para_csv(dados)

    # Separa as variáveis
    dados_list = dados.split(';')
    temperatura = float(dados_list[1])
    umidade = float(dados_list[2])

    # Toma as decisões
    irrigacao = "1" if (20 < temperatura < 30) and (50 < umidade < 70) else "0" #50-70
    semeadura = "1" if (15 < temperatura < 30) and (60 < umidade < 70) else "0" #60-70

    # Envia as decisões tomadas
    data_sender(irrigacao, semeadura)

# Função que lê os dados do broker
def data_reader():
    # Inicializa o cliente MQTT
    client = mqtt.Client()

    # Se inscreve no tópico ao conectar
    def on_connect(client, userdata, flags, rc):
        print("Conectado com código de resultado: " + str(rc))
        client.subscribe("GrupoZ_time_temp_umi_press_alt")

    # Trata os dados recebidos
    def on_message(client, userdata, msg):
        print("Mensagem recebida no tópico {}: {}".format(msg.topic, msg.payload.decode()))
        data_treatment(msg.payload.decode())

    client.on_connect = on_connect
    client.on_message = on_message

    # Dados da conexão
    client.username_pw_set(username, password)
    client.connect(broker_address, port, 60)

    # Tenta se conectar
    try:
        # Entra em loop
        client.loop_forever()
    # Desconecta em caso de erros
    except:
        client.disconnect()
        raise Exception("Desconectando...")

### Coleta de dados do _broker_

Para coletar os lados lidos pelo ESP32, é necessário rodar a célula abaixo pelo período de tempo de coleta desejado. Para interromper a coleta, basta interromper a execução da célula. Ela também é responsável por inicializar o arquivo CSV onde os dados são salvos. Não é necessário executar essa célula caso os dados já estejam em mãos.

In [ ]:
# COLETA DE DADOS

# Se o arquivo CSV não existe, cria
if not os.path.isfile(file_name):
    with open(file_name, 'w', newline='\n') as arquivo_csv:
        print(f"Criando novo arquivo CSV {file_name}")
        arquivo_csv.write("Timestamp;Temperatura;Umidade;Pressao;Altitude\n")
else:
    print(f"Atualizando arquivo CSV {file_name}")

print("\nEncerre a execução da célula para parar de coletar dados!\n")

# Tenta ler os dados recebidos
try:
    data_reader()
except:
    print("Conexão com o broker encerrada!")

### *Download* do arquivo CSV gerado

Não é necessário executar essa célula caso os dados já estejam em mãos, caso contrário, deve-se primeiro escolher o tipo de *download* no menu da célula.
</br>
O tipo `google_drive` se conecta à conta do Google Drive do usuário e salva o arquivo na pasta `caminho_download_gdrive` especificada (deve terminar com o caractere `\`) e com o nome digitado em `nome_download_gdrive`.
</br>
O tipo `direto` exibe um botão para baixar o arquivo na máquina local do usuário.

In [ ]:
# DOWNLOAD DOS DADOS

# Define o menu
tipo_download = 'google_drive' #@param ['direto', 'google_drive'] {allow-input: false}
caminho_download_gdrive = 'Escola/RCOC6/Trabalho 2/' #@param {type: 'string'}
nome_download_gdrive = 'dadosSaida.csv' #@param {type: 'string'}

# Se for um download direto
if tipo_download == 'direto':
    # Salva na máquina
    files.download(file_name)
# Senão, salva no Google Drive
else:
    shutil.copy(file_name, '/drive/My Drive/' + caminho_download_gdrive)

### *Upload* de um arquivo CSV e exibição dos dados

Para exibir os gráficos referentes a um período de coleta, deve-se primeiramente realizar o *upload* de um arquivo CSV. Recomenda-se fortemente carregar o arquivo diretamente do Google Drive, visto que um _upload_ direto é consideravelmente lento. Assim como para o _download_, o carregamento pode ser controlado pela opção `tipo_upload`.
</br>
Se for definida como `google_drive`, o caminho completo até o CSV no Drive do usuário deve ser informado em `caminho_no_gdrive`.
</br>
Já o tipo `direto`permite carregar um arquivo da máquina local através do botão de _upload_, porém, com uma taxa lenta.
</br>
Se o `tipo_upload`for definido como `url`, um URL para o CSV deve ser informado em `url_csv`, e o programa baixará as informações submetidas.
</br>
O parâmetro `periodo` controla qual período de tempo será exibido no gráfico; as opções são auto-explicativas. Se não houver uma quantidade suficiente de dados para o período selecionado, o programa irá exibir uma mensagem de erro.

In [ ]:
# EXIBIÇÃO DE DADOS

# Define o menu
tipo_upload = 'google_drive' #@param ['direto', 'google_drive', 'url'] {allow-input: false}
caminho_no_gdrive = 'Escola/RCOC6/Trabalho 2/dadosTesteFicticio.csv' #@param {type: 'string'}
url_csv = '' #@param {type: 'string'}

# Nome do arquivo carregado
arquivo_upload = ""

# Se for carregado do Google Drive, monta o Drive do usuário e copia o arquivo
if tipo_upload == 'google_drive':
    drive.mount('/drive')
    shutil.copy('/drive/My Drive/' + caminho_no_gdrive, '.')
    arquivo_upload = os.path.basename(caminho_no_gdrive)
# Já se for um upload direto, pega o arquivo carregado
elif tipo_upload == 'direto':
    uploaded = files.upload()
    for fn in uploaded.keys():
        arquivo_upload = fn
# Caso seja informado um URL, baixa o arquivo informado
elif tipo_upload == 'url':
    !wget -nc $url_blend
    arquivo_upload = os.path.basename(url_csv)

# Leitura do arquivo CSV
df = pd.read_csv(arquivo_upload, sep=';').drop(columns=["Altitude"])
# Mostra o head do dataframe
print(df.head())

# Seleção do período para visualização dos dados
#@markdown ---
periodo = "Dias / Ultima semana" #@param ["Segundos / Sempre", "Minutos / Ultima hora", "Horas / Ultimo dia", "Dias / Ultima semana"]

# Dicionário para conversão de segundos para o período selecionado
dict_periodo = {
    "Segundos / Sempre": [1, 60, "Segundos desde um minuto atrás"], # Não é necessário agrupar os dados
    "Minutos / Ultima hora": [60, 60, "Minutos desde uma hora atrás"],
    "Horas / Ultimo dia": [3600, 24, "Horas desde um dia atrás"],
    "Dias / Ultima semana": [86400, 7, "Dias desde uma semana atrás"]
}

# Agrupamento dos dados de acordo com o período selecionado
# Se não houver dados suficientes para o período selecionado, exibe uma mensagem de erro
if df.shape[0] < dict_periodo[periodo][0]*dict_periodo[periodo][1]:
    print("Não há dados suficientes para o período selecionado!")
    exit()
# O agrupamento é feito pela média dos dados de cada período
df_periodo = df.groupby(df.index // dict_periodo[periodo][0]).mean().tail(dict_periodo[periodo][1])
# Remove a coluna "Timestamp"
df_periodo = df_periodo.drop(columns=["Timestamp"])
# Cria uma coluna com o tempo em minutos/horas/dias
df_periodo = df_periodo.reset_index(drop=True)

# Plota os gráficos
fig, axs = plt.subplots(3, 1)
df.plot(y='Temperatura', ax=axs[0], title="Tempo X Temperatura", xlabel="", ylabel="Temperatura (°C)", xlim=[0,dict_periodo[periodo][1]-1], grid=True, xticks=range(0,dict_periodo[periodo][1]), figsize=(25,10))
df.plot(y='Umidade', ax=axs[1], title="Tempo X Umidade", xlabel="", ylabel="Umidade (%Ur)", xlim=[0,dict_periodo[periodo][1]-1], grid=True, xticks=range(0,dict_periodo[periodo][1]), figsize=(25,10))
df.plot(y='Pressao', ax=axs[2], title="Tempo X Pressão", xlabel=dict_periodo[periodo][2], ylabel="Pressão (hPa)", xlim=[0,dict_periodo[periodo][1]-1], grid=True, xticks=range(0,dict_periodo[periodo][1]), figsize=(25,10))
fig_manager = plt.get_current_fig_manager()
plt.show()